In [17]:
import torch
import numpy as np
from escnn import nn, group, gspaces

from models.core.point_convolution import ImplicitPointConv
from utils.utils import get_elu

# Azymuthal-rotation equivariant model

Let's implement a convolutional layer that is equivariant under rotations around the z-axis and acts on field on $\mathbb{R}^3$.

Since we work in escnn, we need to specify the group space, and indicate which subgroup of O(3) we work with.

In [18]:
gspace = gspaces.rot2dOnR3() # SO(2) on R^3
subgroup_id = gspace._sg_id[:-1] # indicator for the subgroup SO(2) \in O(3)

We assume that our input is 3 vector fields and 2 scalar fields, and the output is 1 vector field.

In [ ]:
# restrict the standard representation of O(3) to SO(2)
std_repr = group.o3_group().standard_representation().restrict(subgroup_id) 
triv_repr = gspace.trivial_repr 

in_repr = 3*[std_repr] + 2*[triv_repr]
out_repr = 1*[std_repr] + 1*[triv_repr]

# set field type of the input and output
in_type = gspace.type(*in_repr)
out_type = gspace.type(*out_repr)

Implicit point convolution takes as input node and edge features of a geometric graph.

Hence, we have to specify the representation of edge features.

Let us assume that we have 2 edge features,  one of which is a scalar field and one is a vector field.

In [29]:
edge_repr = 1*[gspaces.no_base_space(group.o3_group()).trivial_repr] + 1*[group.o3_group().standard_representation()]

[OPTIONAL] For better initialization, we can give an approximate feature distribution to the kernel.

First element is for relative positions, second is for additional edge features specified above.

Assuming that edge features follow a normal distribution with mean 0 and std 0.5, we have:

In [30]:
edge_distr = [None, torch.distributions.Normal(torch.zeros(4), 0.5*torch.ones(4))]

We also need to specify the order of harmonic polynomials we use in the implicit kernel.

Let's use polynomials of order 2.

In [20]:
hp_order = 2

Last, we need to specify parameters of the MLP with which we parametrize steerable filters.

In [ ]:
mlp_params = dict(n_layers=3, 
                  n_channels=8, 
                  act_fn='elu', 
                  use_tp=False)

Let us now build a Steerable CNN model with 3 convolutional layers and QuotientFourier non-linearity.

It is important to say that implicit kernels only support uniform representations.

It means that the input and output representations of the model must be the copies of the same representation.

This is not a limitatiom per se, since we can always map a non-uniform representation to a uniform one, e.g. using a Projector module.

In [21]:
class Projector(nn.EquivariantModule):
    def __init__(self, in_type: nn.FieldType, out_type: nn.FieldType):
        super().__init__()
        G = in_type.gspace.fibergroup
        gspace = gspaces.no_base_space(G)
        self.in_type = in_type
        self.hid_type1 = gspace.type(*in_type.representations)
        self.hid_type2 = gspace.type(*out_type.representations)
        self.linear = nn.Linear(self.hid_type1, self.hid_type2) 
        self.out_type = out_type
    
    def forward(self, x):
        x, coords = x.tensor, x.coords
        x = self.hid_type1(x)
        x = self.linear(x)
        x = nn.GeometricTensor(x.tensor, self.out_type, coords)
        return x
    
    def evaluate_output_shape(self):
        pass

In [22]:
# We use 16 hidden channels for all layers and band-limit representations up to frequency L=1
hidden_channels = 3
L = 1

activation = get_elu(gspace = in_type.gspace, L = L, channels = hidden_channels)

# in Steerable CNNs, hidden channels are determined by the activation function.
hidden_type = activation.out_type
print(f"{hidden_channels} hidden fields with representation: {hidden_type.representations[0]}")

proj_in = Projector(in_type, hidden_type)

layer1 = ImplicitPointConv(
    in_type=hidden_type,
    out_type=hidden_type,
    edge_repr=edge_repr,
    hp_order=hp_order,
    edge_distr=edge_distr,
    **mlp_params)

layer2 = ImplicitPointConv(
    in_type=hidden_type,
    out_type=hidden_type,
    edge_repr=edge_repr,
    hp_order=hp_order,
    edge_distr=edge_distr,
    **mlp_params)

layer3 = ImplicitPointConv(
    in_type=hidden_type,
    out_type=hidden_type,
    edge_repr=edge_repr,
    hp_order=hp_order,
    edge_distr=edge_distr,
    **mlp_params)

proj_out = Projector(hidden_type, out_type)

3 hidden fields with representation: SO(2)|[regular_[(0,)|(1,)]]:3


In [23]:
x = nn.GeometricTensor(torch.randn(10,11), in_type, torch.randn(10,3))
edge_index = torch.randint(0, 10, (2, 20))
edge_delta = torch.randn(20,3)
edge_attr = torch.randn(20,4)

x = proj_in(x)
x = layer1(x=x, edge_index=edge_index, edge_delta=edge_delta, edge_attr=edge_attr, idx_downsampled=None)
x = activation(x)
x = layer2(x=x, edge_index=edge_index, edge_delta=edge_delta, edge_attr=edge_attr, idx_downsampled=None)
x = activation(x)
x = layer3(x=x, edge_index=edge_index, edge_delta=edge_delta, edge_attr=edge_attr, idx_downsampled=None)
x = activation(x)
x = proj_out(x)

In [31]:
print(x)

g_tensor([[-1.6786,  0.7669,  0.1886,  0.3077,  1.3473,  1.4544,  1.4313,  0.4587,
           -0.1433],
          [ 0.5265,  0.2096, -1.2069,  0.1216,  0.1228,  1.8395, -0.3537,  0.2657,
           -1.4052],
          [-0.7541, -1.5691,  1.0889, -0.7446,  0.6736,  1.4426, -0.3475, -0.4107,
           -0.4201],
          [-0.0606,  0.7885, -0.6417,  0.0520,  0.7724, -0.8386,  0.4745, -0.3549,
            0.4354],
          [-0.2242, -0.6032, -0.8123, -1.8604,  1.1409,  0.9630, -0.4661, -0.3021,
            0.7514],
          [ 0.8057,  0.0545, -1.2078, -1.1823,  0.3829, -0.3940, -0.5657,  0.8113,
           -1.3076],
          [ 1.1761, -1.1030,  1.4868,  0.4862,  1.9211,  1.4776,  1.9929,  1.1195,
           -0.8925],
          [ 0.2114,  2.1323, -0.4631, -0.3451,  0.3074, -0.3687, -1.8141,  0.4664,
           -1.3572],
          [-0.7152, -2.8481,  0.0283, -0.8389,  0.8672, -0.4120,  0.1094,  0.7757,
           -0.2128],
          [-0.5213,  0.5629, -0.1977, -0.4839, -0.2761, -1.0859,

# Test equivariance

We need to specify the type of edge features for the layer (it is done automatically inside the implicit kernel)

In [25]:
std_type = gspaces.no_base_space(group.o3_group()).type(*[group.o3_group().standard_representation()]).restrict(subgroup_id) 
edge_type = gspaces.no_base_space(group.o3_group()).type(*edge_repr).restrict(subgroup_id) 

In [26]:
layer_to_test = layer1

In [32]:
x = nn.GeometricTensor(torch.randn(10,hidden_type.size), layer_to_test.in_type, torch.randn(10,3))

errors = []

for el in gspace.testing_elements:
    out1 = layer_to_test(x=x, edge_index=edge_index, edge_delta=edge_delta, edge_attr=edge_attr, idx_downsampled=None).transform_fibers(el).tensor.detach().numpy()

    edge_delta_ = nn.GeometricTensor(edge_delta, std_type)
    edge_attr_ = nn.GeometricTensor(edge_attr, edge_type)
    out2 = layer_to_test(x.transform_fibers(el), 
                  edge_index=edge_index, 
                  edge_delta=edge_delta_.transform_fibers(el).tensor, 
                  edge_attr=edge_attr_.transform_fibers(el).tensor, 
                  idx_downsampled=None).tensor.detach().numpy()

    errs = np.abs(out1 - out2)
    errors.append(errs.mean())

print(f"Average absolute error for the layer: {np.mean(errors):.2e}")

Average absolute error for the layer: 1.42e-04


The absolute error is higher that machine epsilon since we use quotient fourier nonlinearities inside of the $G$-MLP, which involves discretization and hence brings error.
It is however gives us leverage on the degree of equivariance we want to have in our model.